In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator 
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization
from keras.optimizers import Adam
from keras.layers import Activation
from os import listdir
from numpy import asarray,save
from keras.utils import load_img, img_to_array
import random
from keras import backend as K

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = ImageDataGenerator(rescale =1.0/255.0, rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,vertical_flip=True,
                         horizontal_flip=True, fill_mode="nearest")
test = ImageDataGenerator(rescale=1.0/255.0, rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,vertical_flip=True,
                         horizontal_flip=True, fill_mode="nearest")

In [ ]:
# initial parameters
epochs = 100
lr = 1e-3
batch_size = 64
img_dims = (96,96,3)

In [ ]:
train_data = train.flow_from_directory('/content/drive/MyDrive/Gender and Age Detection Final Project Data/Age Dataset Main Final Project/Age Detection', target_size=(96,96), class_mode='categorical')
test_data = test.flow_from_directory('/content/drive/MyDrive/Gender and Age Detection Final Project Data/Age Dataset Main Final Project/Storage Test', target_size=(96,96), class_mode='categorical')
print('train_index :', train_data.class_indices)
print('test_index:', test_data.class_indices)

Found 5080 images belonging to 10 classes.
Found 100 images belonging to 10 classes.
train_index : {'eightyone_to_ninety': 0, 'eleven_to_twenty': 1, 'fiftyone_to_sixty': 2, 'fourtyone_to_fifty': 3, 'ninetyone_to_onehundred': 4, 'seventyone_to_eighty': 5, 'sixtyone_to_seventy': 6, 'thirtyone_to_fourty': 7, 'twentyone_to_thirdty': 8, 'zero_to_ten': 9}
test_index: {'eightyone_to_ninety': 0, 'eleven_to_twenty': 1, 'fiftyone_to_sixty': 2, 'fourtyone_to_fifty': 3, 'ninetyone_to_onehundred': 4, 'seventyone_to_eighty': 5, 'sixtyone_to_seventy': 6, 'thirtyone_to_fourty': 7, 'twentyone_to_thirdty': 8, 'zero_to_ten': 9}


In [ ]:
model= Sequential()
chanDim = -1
if K.image_data_format() == "channels_first": #Returns a string, either 'channels_first' or 'channels_last'
    chanDim = 1
#CNN1
model.add(Conv2D(32, kernel_size=(3,3), activation='relu',input_shape=(96,96,3), padding='same'))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3,3), padding='same'))
model.add(Dropout(0.25))
#CNN2
model.add(Conv2D(64,(3,3),activation='relu', padding='same'))
model.add(BatchNormalization(axis=chanDim))
#CNN3
model.add(Conv2D(128,(3,3),activation='relu', padding='same'))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.25))  
#CNN4
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', padding="same"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.25))

batch_size=128
epochs =100
classes = 10

In [ ]:
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(classes))
model.add(Activation("sigmoid"))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 96, 96, 32)        896       
                                                                 
 batch_normalization_17 (Bat  (None, 96, 96, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 32, 32, 32)       0         
 g2D)                                                            
                                                                 
 dropout_14 (Dropout)        (None, 32, 32, 32)        0         
                                                                 
 conv2d_15 (Conv2D)          (None, 32, 32, 64)        18496     
                                                                 
 batch_normalization_18 (Bat  (None, 32, 32, 64)      

In [ ]:
from keras.optimizers import Adam
opt = Adam(learning_rate=lr, decay=lr/epochs)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 96, 96, 32)        896       
                                                                 
 batch_normalization_17 (Bat  (None, 96, 96, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 32, 32, 32)       0         
 g2D)                                                            
                                                                 
 dropout_14 (Dropout)        (None, 32, 32, 32)        0         
                                                                 
 conv2d_15 (Conv2D)          (None, 32, 32, 64)        18496     
                                                                 
 batch_normalization_18 (Bat  (None, 32, 32, 64)      

In [ ]:
his=model.fit(train_data,epochs=epochs,batch_size=128,verbose=1,validation_data=test_data)
value=model.evaluate(train_data,verbose=0)
print('loss', value[0])
print('accuracy', value[1])

Epoch 1/100
159/159 [==============================] - 54s 334ms/step - loss: 0.5466 - accuracy: 0.2183 - val_loss: 2.0010 - val_accuracy: 0.1000
Epoch 2/100
159/159 [==============================] - 26s 164ms/step - loss: 0.3207 - accuracy: 0.3104 - val_loss: 3.2244 - val_accuracy: 0.1000
Epoch 3/100
159/159 [==============================] - 26s 164ms/step - loss: 0.3001 - accuracy: 0.3463 - val_loss: 1.3650 - val_accuracy: 0.0800
Epoch 4/100
159/159 [==============================] - 26s 166ms/step - loss: 0.2958 - accuracy: 0.3348 - val_loss: 0.5369 - val_accuracy: 0.1600
Epoch 5/100
159/159 [==============================] - 27s 167ms/step - loss: 0.2807 - accuracy: 0.3644 - val_loss: 0.7461 - val_accuracy: 0.1600
Epoch 6/100
159/159 [==============================] - 26s 166ms/step - loss: 0.2714 - accuracy: 0.3754 - val_loss: 0.3085 - val_accuracy: 0.2500
Epoch 7/100
159/159 [==============================] - 26s 163ms/step - loss: 0.2652 - accuracy: 0.3937 - val_loss: 0.3218 -

In [ ]:
model.save('/content/drive/MyDrive/Model Save/Age_Detection.h5')